In [164]:
import requests
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

# Get data on overall results using BeautifulSoup

In [8]:
x = requests.get('https://www.pgatour.com/leaderboard/hole-by-hole')
print(x.status_code)

200


In [9]:
soup = BeautifulSoup(x.content, 'lxml')

In [15]:
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0"
}

In [50]:
#print(soup.prettify())

In [51]:
#soup.find_all('script')

In [247]:
# Convert JSON data into Python dictionary
all_data = soup.find_all("script", {"type": "application/json"})

for data in all_data:
    jsn = json.loads(data.string)
  ##  print(json.dumps(jsn, indent=4))

In [249]:
# Extract individual player data from dictionary 
player_dicts = jsn["props"]["pageProps"]["leaderboard"]["players"]

In [190]:
# Extract player information using list comprehension
# Add if condition to remove cut line entry mid table
names = [player_dicts[elem]['player']['displayName'] for elem in range(len(player_dicts)) if 'player' in player_dicts[elem]]
countries = [player_dicts[elem]['player']['country'] for elem in range(len(player_dicts)) if 'player' in player_dicts[elem]]
ids = [player_dicts[elem]['player']['id'] for elem in range(len(player_dicts)) if 'player' in player_dicts[elem]]

# Extract scoring information using list comprehension
positions = [player_dicts[elem]['scoringData']['position'] for elem in range(len(player_dicts)) if 'scoringData' in player_dicts[elem]]
total_scores = [player_dicts[elem]['scoringData']['total'] for elem in range(len(player_dicts)) if 'scoringData' in player_dicts[elem]]
rounds = [player_dicts[elem]['scoringData']['rounds'] for elem in range(len(player_dicts)) if 'scoringData' in player_dicts[elem]]
total_strokes = [player_dicts[elem]['scoringData']['totalStrokes'] for elem in range(len(player_dicts)) if 'scoringData' in player_dicts[elem]]
round_status = [player_dicts[elem]['scoringData']['roundStatus'] for elem in range(len(player_dicts)) if 'scoringData' in player_dicts[elem]]

# Fill missing status values for players that withdrew
round_status[-3:] = ["Withdrew","Withdrew","Withdrew"]

# Extract individual round scores
r1 = [rounds[elem][0] for elem in range(len(rounds))]
r2 = [rounds[elem][1] for elem in range(len(rounds))]
r3 = [rounds[elem][2] for elem in range(len(rounds))]
r4 = [rounds[elem][3] for elem in range(len(rounds))]

In [94]:
player_dicts[0]

{'__typename': 'PlayerRowV3',
 'id': '48081',
 'leaderboardSortOrder': 0,
 'player': {'id': '48081',
  'firstName': 'Xander',
  'lastName': 'Schauffele',
  'amateur': False,
  'displayName': 'Xander Schauffele',
  'abbreviations': '',
  'abbreviationsAccessibilityText': '',
  'country': 'USA',
  'countryFlag': 'USA',
  'shortName': 'X. Schauffele',
  'lineColor': '#0084FF',
  'tourBound': False},
 'scoringData': {'position': '1',
  'total': '-21',
  'totalSort': -21,
  'thru': 'F',
  'thruSort': 19,
  'score': '-6',
  'scoreSort': -6,
  'courseId': '671',
  'groupNumber': 39,
  'currentRound': 4,
  'backNine': False,
  'roundHeader': 'R4',
  'rounds': ['62', '68', '68', '65'],
  'movementDirection': 'CONSTANT',
  'movementAmount': '0',
  'playerState': 'COMPLETE',
  'rankingMovement': 'CONSTANT',
  'rankingMovementAmount': '-',
  'rankingMovementAmountSort': 0,
  'rankLogoLight': 'https://res.cloudinary.com/pgatour-prod/temp/standings/profile/Comcast-Light.png',
  'rankLogoDark': 'http

# Get hole-by-hole data using Selenium

In [148]:
driver = webdriver.Edge()
driver.get('https://www.pgatour.com/tournaments/2024/pga-championship/R2024033/leaderboard/hole-by-hole')
# print("Page title is: %s" %(driver.title))

In [214]:
r1_holes = [driver.find_element(By.XPATH, f'//tr[@class="player-{elem} css-paaamq"]').text for elem in ids]
r1_split = pd.Series(r1_holes).apply(lambda x: x.split('\n'))

In [215]:
r2_holes = [driver.find_element(By.XPATH, f'//tr[@class="player-{elem} css-paaamq"]').text for elem in ids]
r2_split = pd.Series(r2_holes).apply(lambda x: x.split('\n'))

In [216]:
r3_holes = [driver.find_element(By.XPATH, f'//tr[@class="player-{elem} css-paaamq"]').text for elem in ids]
r3_split = pd.Series(r3_holes).apply(lambda x: x.split('\n'))

In [203]:
r4_holes = [driver.find_element(By.XPATH, f'//tr[@class="player-{elem} css-paaamq"]').text for elem in ids]
r4_split = pd.Series(r4_holes).apply(lambda x: x.split('\n'))

In [246]:
print(r4_split[0][3:12],r4_split[0][14:22])

['3', '4', '3', '3', '4', '4', '4', '3', '3'] ['2', '3', '4', '3', '4', '4', '4', '4']


In [251]:
r1_scores = [r1_split[elem][3:12]+r1_split[elem][13:22] for elem in range(len(r4_split))] 
r2_scores = [r2_split[elem][3:12]+r2_split[elem][13:22] for elem in range(len(r4_split))] 
r3_scores = [r3_split[elem][3:12]+r3_split[elem][13:22] for elem in range(len(r4_split))] 
r4_scores = [r4_split[elem][3:12]+r4_split[elem][13:22] for elem in range(len(r4_split))] 

In [268]:
holes = ["hole "+ f"{elem}" for elem in range(1,19)]

In [278]:
r1_scores[0]

['4',
 '3',
 '3',
 '3',
 '3',
 '4',
 '4',
 '3',
 '4',
 '5',
 '2',
 '4',
 '3',
 '3',
 '3',
 '3',
 '4',
 '4']

In [269]:
holes

['hole 1',
 'hole 2',
 'hole 3',
 'hole 4',
 'hole 5',
 'hole 6',
 'hole 7',
 'hole 8',
 'hole 9',
 'hole 10',
 'hole 11',
 'hole 12',
 'hole 13',
 'hole 14',
 'hole 15',
 'hole 16',
 'hole 17',
 'hole 18']

# Create master df

In [250]:
# Create principal df using dictionary
pga_dict = {'Name': names, 'Country': countries, 'Position': positions, 'Total score': total_scores, 'R1': r1, 'R2': r2, 'R3': r3, 'R4': r4, 'Total strokes': total_strokes, 'Status' : round_status}
df = pd.DataFrame(pga_dict)
df

,Name,Country,Position,Total score,R1,R2,R3,R4,Total strokes,Status
0,Xander Schauffele,USA,1,-21,62,68,68,65,263,R4 Completed
1,Bryson DeChambeau,USA,2,-20,68,65,67,64,264,R4 Completed
2,Viktor Hovland,NOR,3,-18,68,66,66,66,266,R4 Completed
3,Thomas Detry,BEL,T4,-15,66,67,70,66,269,R4 Completed
4,Collin Morikawa,USA,T4,-15,66,65,67,71,269,R4 Completed
...,...,...,...,...,...,...,...,...,...,...
151,Jeff Kellen,USA,CUT,+17,87,72,-,-,159,R2 Completed
152,Rich Beem,USA,CUT,+20,79,83,-,-,162,R2 Completed
153,John Daly,USA,WD,-,82,-,-,-,82,Withdrew
154,Ben Griffin,USA,WD,-,73,65,-,-,138,Withdrew


In [291]:
pga_dict['Name'][8]

'Scottie Scheffler'

# Hole by hole df

In [ ]:
# holes[0] : r1_scores[elem][0]

In [ ]:
hbh_r1 = 

In [ ]:
def hbh_zipper(scores_list):
    holes = ["hole "+ f"{elem}" for elem in range(1,19)]
    for elem in range(len(scores_list):    
        h_scores = [scores_list[elem][0] for elem in scores_list]
    

In [279]:
r1_scores[0]

['4',
 '3',
 '3',
 '3',
 '3',
 '4',
 '4',
 '3',
 '4',
 '5',
 '2',
 '4',
 '3',
 '3',
 '3',
 '3',
 '4',
 '4']

In [289]:
holes_dict = {"hole "+str(hole): [r1_scores[player][hole-1] for player in range(len(r1_scores))] for hole in range(1,19)}
df_holes = pd.DataFrame(holes_dict)
df_holes

,hole 1,hole 2,hole 3,hole 4,hole 5,hole 6,hole 7,hole 8,hole 9,hole 10,hole 11,hole 12,hole 13,hole 14,hole 15,hole 16,hole 17,hole 18
0,4,3,3,3,3,4,4,3,4,5,2,4,3,3,3,3,4,4
1,4,4,3,4,4,4,3,2,4,5,3,5,4,3,3,4,4,5
2,4,4,3,4,4,4,4,3,4,4,3,3,3,4,4,3,5,5
3,4,3,3,4,5,4,4,3,4,4,3,4,3,3,4,4,3,4
4,4,5,3,4,5,3,4,2,4,5,3,3,3,2,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,5,4,4,4,5,6,6,3,5,5,4,5,4,4,7,5,5,6
152,5,6,4,5,4,4,5,3,4,5,3,5,3,3,4,6,5,5
153,5,5,4,4,5,5,5,4,5,5,3,5,4,4,4,4,6,5
154,4,3,3,5,3,4,5,3,4,5,3,3,6,4,4,5,5,4
